# Figure 1

Start by loading some boiler plate: matplotlib, numpy, scipy, json, functools, and a convenience class.

In [1]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d, InterpolatedUnivariateSpline
from scipy.optimize import bisect
import json
from functools import partial
class Foo: pass

And some more specialized dependencies:
 1. ``Slict`` provides a convenient slice-able dictionary interface
 2. ``Chest`` is an out-of-core dictionary that we'll hook directly to a globus remote using...
 3. ``glopen`` is an open-like context manager for remote globus files

In [2]:
from chest import Chest
from slict import CachedSlict
from glopen import glopen, glopen_many

Configuration for this figure.

In [3]:
config = Foo()
config.name0     = "Wilk/Wilk_kmin_2.5/Wilk_kmin_2.5"
config.name1     = "Wilk/Wilk_kmin_4.5/Wilk_kmin_4.5"
#config.arch_end = "maxhutch#alpha-admin/~/pub/"
config.arch_end = "alcf#dtn_mira/projects/alpha-nek/"

Open a chest located on a remote globus endpoint and load a remote json configuration file.

In [4]:
c0 = Chest(path      = "{:s}-results".format(config.name0),
          open      = partial(glopen,      endpoint=config.arch_end),
          open_many = partial(glopen_many, endpoint=config.arch_end))
sc0 = CachedSlict(c0)
with glopen(
            "{:s}.json".format(config.name0), mode='r',
            endpoint = config.arch_end,
            ) as f:
    params0 = json.load(f)

In [5]:
c1 = Chest(path      = "{:s}-results".format(config.name1),
          open      = partial(glopen,      endpoint=config.arch_end),
          open_many = partial(glopen_many, endpoint=config.arch_end))
sc1 = CachedSlict(c1)
with glopen(
            "{:s}.json".format(config.name1), mode='r',
            endpoint = config.arch_end,
            ) as f:
    params1 = json.load(f)

We want to plot the spike depth, which is the 'H' field in the chest.
Chests can prefetch lists of keys more quickly than individual ones, so we'll prefetch the keys we want.

In [6]:
c0.prefetch(sc0[:,'H'].full_keys())
c0.prefetch(sc0[:,'w_max_z'].full_keys())
c1.prefetch(sc1[:,'H'].full_keys())
c1.prefetch(sc1[:,'w_max_z'].full_keys())

KeyboardInterrupt: 

Plot the bubble height, the 'H' keys, vs. time.
Use a spline to compute the derivative.

In [ ]:
spl0 = InterpolatedUnivariateSpline(sc0[:,'H'].keys(),
                                    sc0[:,'H'].values() / np.sqrt(params0["atwood"]*params0["g"] / params0["kmin"]),
                                    k=3)
Fr0 = spl0.derivative()
Ts0 = np.linspace(sc0[:,'H'].keys()[0], sc0[:,'H'].keys()[-1], 1000)

In [ ]:
spl1 = InterpolatedUnivariateSpline(sc1[:,'H'].keys(), 
                                    sc1[:,'H'].values()/ np.sqrt(params1["atwood"]*params1["g"] / params1["kmin"]),
                                    k=3)
Fr1 = spl1.derivative()
Ts1 = np.linspace(sc1[:,'H'].keys()[0], sc1[:,'H'].keys()[-1], 1000)

In [ ]:
fig, axs = plt.subplots(2,1, sharex=True)

axs[0].plot(Ts0, -spl0(Ts0));
axs[0].plot(Ts1, -spl1(Ts1));
#axs[0].plot(sc[:,'H'].keys(), -np.array(sc[:,'H'].values()), 'yo');
axs[0].set_ylabel('Depth (m)')

axs[1].plot(Ts0, -Fr0(Ts0), 'b-');
axs[1].plot(Ts1, -Fr1(Ts1), 'b-');
axs[1].plot([0,.45], [np.sqrt(1/np.pi), np.sqrt(1/np.pi)], 'k--')
axs[1].set_ylabel('Velocity (scaled)')
axs[1].set_xlabel('Time (s)');
plt.savefig('Figure1.eps')

In [ ]:
%install_ext http://raw.github.com/jrjohansson/version_information/master/version_information.py 
%load_ext version_information 
%version_information numpy, matplotlib, slict, chest, glopen, globussh